In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

In [ ]:
# parameter settings
is_plot = True
is_export = True

landmarks_path = '../data/landmarks/refine_6kmh_braless_18markers_12fps.pkl'
meshes_path = '../data/meshes/6kmh_braless_26markers/'
test_landmarks_path = '../data/test/braless_random_landmarks.pkl'

start=0
stride = 12
end=120

# Data Loading

In [ ]:
from mesh4d import obj3d
from mesh4d.analyse import crave

mesh_ls, texture_ls = obj3d.load_mesh_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
)
mesh_ls = [crave.fix_pvmesh_disconnect(mesh) for mesh in mesh_ls]

In [ ]:
from mesh4d import utils

landmarks = utils.load_pkl_object(landmarks_path)
landmarks.interp_field()

In [ ]:
from mesh4d.analyse.crave import clip_with_contour

contour = landmarks.extract(('marker 0', 'marker 2', 'marker 15', 'marker 17'))
mesh_clip_ls = clip_with_contour(mesh_ls, start_time=0, fps=12, contour=contour, clip_bound='xy', margin=0.5)

In [ ]:
body_ls = obj3d.init_obj_series(
    mesh_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

In [ ]:
breast_ls = obj3d.init_obj_series(
    mesh_clip_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

---
# Registration Approach Selection


_P.S. Only activate one of the following code blocks in this section_

In [ ]:
# """
from mesh4d.regist import reg_rbf

obj3d_ls = body_ls
obj4d_type = reg_rbf.Obj4d_RBF
export_folder = 'output/rbf/'
# """

In [ ]:
"""
from mesh4d.regist import reg_ecpd

obj3d_ls = breast_ls
obj4d_type = reg_ecpd.Obj4d_ECPD
export_folder = 'output/ecpd/'
"""

# Ultra-dense Motion Capture

In [ ]:
import time
start_time = time.time()

o4 = obj4d_type(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=True,
)
o4.add_obj(*obj3d_ls)
o4.load_markerset('landmarks', landmarks)
o4.regist('landmarks')

duration = time.time() - start_time

In [ ]:
print("computation time: {}".format(duration))

if is_export:
    utils.save_pkl_object(duration, export_folder=export_folder, export_name='duration')

## Control Landmarks

In [ ]:
kps_source = landmarks.get_time_coord(0)
o4.vkps_track(kps_source, start_id=0)

In [ ]:
from mesh4d import kps
vkps = o4.assemble_markerset(name='vkps')
diff = kps.MarkerSet.diff(vkps, landmarks)

In [ ]:
if is_export:
    utils.save_pkl_object(diff, export_folder, 'diff')

In [ ]:
if is_plot:
    o4.export_pcd_gif(export_folder, filename='vkps', kps_names=('vkps', 'landmarks'))

In [ ]:
import numpy as np
from mesh4d.analyse import visual

error_ls = []

for marker in diff['diff_dict'].values():
    error_ls.append(marker['dist_mean'])

error_ls = np.array(error_ls)

if is_plot:
    landmarks_init = landmarks_init = landmarks.get_frame_coord(1).get_points_coord()
    visual.show_mesh_value_mask(mesh_clip_ls[1], landmarks_init, error_ls/10, k_nbr=100, show_edges=True)

## Randomly Selected Landmarks

In [ ]:
from mesh4d import utils

random_landmarks = utils.load_pkl_object(test_landmarks_path)
random_kps = random_landmarks.get_frame_coord(0)

In [ ]:
o4.vkps_track(random_kps, start_id=0, name='vkps_random')
vkps_random = o4.assemble_markerset(name='vkps_random')
vkps_random.interp_field()

In [ ]:
from mesh4d.obj4d import Obj4d_Kps

if is_plot:
    o4_breast = Obj4d_Kps(fps=120 / stride)
    o4_breast.add_obj(*breast_ls)
    o4_breast.load_markerset('vkps_random', vkps_random)
    o4_breast.export_pcd_gif(output_folder=export_folder, filename='vkps_random', kps_names=('vkps_random',))

In [ ]:
import os
import mesh4d
import pyvista as pv

if is_export:
    scene = pv.Plotter()
    vkps_random.add_to_scene(scene)

    export_path = os.path.join(export_folder, 'trace.png')
    scene.camera_position = 'xy'
    scene.show(screenshot=export_path)

    if mesh4d.output_msg:
        print("export image: {}".format(export_path))

In [ ]:
import os
import mesh4d
import pyvista as pv

if is_plot:
    scene = pv.Plotter()
    vkps_random.add_to_scene(scene)
    breast_ls[0].add_mesh_to_scene(scene, point_size=1e-3, opacity=0.1)
    scene.camera_position = 'xy'
    scene.show()

## Full-field Breast Deformation Estimation

In [ ]:
breast_kps = breast_ls[1].get_sample_kps(1000)
o4.vkps_track(breast_kps, start_id=1, name='vkps_breast_full')

In [ ]:
vkps_breast_full = o4.assemble_markerset(name='vkps_breast_full', start_id=1)
vkps_breast_full.interp_field()

In [ ]:
from mesh4d.analyse import visual, measure

_, starts, traces = measure.markerset_trace_length(vkps_breast_full, start_frame=1)

if is_plot:
    visual.show_mesh_value_mask(
        mesh_clip_ls[1], starts, np.array(traces)/10,
        is_save=True, export_folder=export_folder, export_name='breast_disp',
        show_edges=True)